In [1]:
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [34]:
graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()
    global_step = tf.train.get_or_create_global_step()
    init_lr = tf.constant(value=1e-5, shape=[], dtype=tf.float32)
    
    is_warm = tf.placeholder(tf.int32, [])
    warmup_step = 20
    warmup_percent_done = tf.cast(global_step, tf.int32) / tf.cast(tf.constant(warmup_step), tf.int32)
    warmup_learning_rate = init_lr * tf.cast(warmup_percent_done, tf.float32)
    
    decay_global_step = tf.cond(tf.cast(warmup_percent_done, tf.float32) < tf.cast(tf.constant(1), tf.float32),
                               lambda:tf.cast(tf.constant(0), tf.int64),
                               lambda:tf.cast(global_step, tf.int64)-tf.cast(tf.constant(warmup_step), tf.int64))
    learning_rate = tf.train.polynomial_decay(
                                                init_lr,
                                                decay_global_step,
                                                100000,
                                                end_learning_rate=0.0,
                                                power=1.0,
                                                cycle=False)
    
    learned_rate = tf.cond(tf.cast(warmup_percent_done, tf.float32) < tf.cast(tf.constant(1), tf.float32),
                          lambda:warmup_learning_rate,
                          lambda:learning_rate)
    
#     learned_rate = tf.cast(1-is_warm, tf.float32)*learning_rate + tf.cast(is_warm, tf.float32) * (warmup_learning_rate)
    
    new_global_step = global_step + 1
    op = global_step.assign(new_global_step)
    sess.run(tf.global_variables_initializer())
    for l in range(21):
        [_, act_lr, warmup_lr, warmup_percent_done_] = sess.run([op, learned_rate, warmup_learning_rate, warmup_percent_done],
                                                               feed_dict={is_warm:0})
        print(act_lr, warmup_lr,warmup_percent_done_, "===warming up===", l)
    for l in range(10):
        [_, decay_lr, act_lr, warmup_lr] = sess.run([op, learning_rate, learned_rate, warmup_learning_rate], feed_dict={is_warm:1})
        print(decay_lr, act_lr, warmup_lr, "===learning rate decay===")

0.0 0.0 0.0 ===warming up=== 0
5e-07 5e-07 0.05 ===warming up=== 1
1e-06 1e-06 0.1 ===warming up=== 2
1.5e-06 1.5e-06 0.15000000000000002 ===warming up=== 3
2e-06 2e-06 0.2 ===warming up=== 4
2.5e-06 2.5e-06 0.25 ===warming up=== 5
3e-06 3e-06 0.30000000000000004 ===warming up=== 6
3.4999998e-06 3.4999998e-06 0.35000000000000003 ===warming up=== 7
4e-06 4e-06 0.4 ===warming up=== 8
4.4999997e-06 4.4999997e-06 0.45 ===warming up=== 9
5e-06 5e-06 0.5 ===warming up=== 10
5.5e-06 5.5e-06 0.55 ===warming up=== 11
6e-06 6e-06 0.6000000000000001 ===warming up=== 12
6.4999995e-06 6.4999995e-06 0.65 ===warming up=== 13
7.5e-06 7.5e-06 0.75 ===warming up=== 14
8e-06 8e-06 0.8 ===warming up=== 15
8e-06 8e-06 0.8 ===warming up=== 16
8.999999e-06 8.999999e-06 0.9 ===warming up=== 17
8.999999e-06 8.999999e-06 0.9 ===warming up=== 18
9.5e-06 9.5e-06 0.9500000000000001 ===warming up=== 19
9.9999e-06 1e-05 1.0 ===warming up=== 20
9.9998e-06 9.9998e-06 1.0499999e-05 ===learning rate decay===
9.9997e-06 

In [20]:
0.1/4

0.025